In [1]:
import datetime
import time
import master

import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.join(os.path.dirname('.'), '..', 'lib'))
sys.path.insert(0, '/Users/chris/programming/interactivebrokers')
import ibapi

In [2]:
import marketdata
import contracts
import helper

PORT = 7496
md_app = marketdata.get_instance(port=PORT)
ct_app = contracts.get_instance(port=PORT)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


serverVersion:155 connectionTime:b'20200410 11:21:30 EST'
MarketDataApp connecting to IB...
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


MarketDataApp connected.
serverVersion:155 connectionTime:b'20200410 11:21:30 EST'
MarketDataApp connecting to IB...
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
MarketDataApp connected.


In [3]:
def download_and_save_daily_data(md_app, contract, start, end, skip_weekends=True, 
                                             data_type='TRADES', min_count=1500):
    period_start = start
    while period_start < end:
        # Skip weekends
        if period_start.weekday() in [5, 6] and skip_weekends:
            period_start += datetime.timedelta(days=1)
            continue
            
        # Get the next period end            
        period_end = min(end, period_start + datetime.timedelta(days=1))

        # Get the request object
        tws_start = helper.convert_datetime_to_tws_date(period_start)
        tws_end = helper.convert_datetime_to_tws_date(period_end)    
        reqObjList = md_app.create_historical_data_request([contract], is_snapshot=True, frequency=frequency, 
                                                use_rth=False, data_type=data_type,
                                                start=tws_start, end=tws_end, duration="")
        reqObj = reqObjList[0]
        assert reqObj is not None, 'Must be legitimate request'

        # Download the data for the next period
        df = download_daily_data(reqObj, start=tws_start, end=tws_end, frequency=frequency, min_count=min_count)

        # Save to file    
        date = tws_start.split(' ')[0]
        filename = 'data/{}_{}_{}.csv'.format(ticker, reqObj.frequency, date)
        df.to_csv(filename, index=False)
        print('Saved data to {}'.format(filename))

        # Get the next start date
        period_start += datetime.timedelta(days=1)
    
    
def download_daily_data(reqObj, start, end, frequency, data_type='TRADES', min_count=1500):    

    # Place the time series request
    reqObj.place_request()
    
    # Wait until the data has been returned by IB
    data = reqObj.get_data()
    n_requests = len(reqObj.get_subrequests())

    t0 = time.time()
    ct = 0
    while ct != 48:
        print('{}: {}'.format(int(time.time() - t0), ct))
        time.sleep(5)
        ct = sum([min_count <= r for r in [len(data[idx][0]) for idx in range(n_requests)]])

    print('Finished downloading for {}. Time elapsed: {}'.format(start, time.time() - t0))
    
    # Turn the data into a dataframe
    df = pd.DataFrame()
    for d in data:
        df = pd.concat([df, pd.DataFrame(d[0])])

    # Drop duplicated rows
    df.drop_duplicates(inplace=True)
    df = df[df.volume > 0]
    return df

In [4]:
partial_contract = ibapi.contract.Contract()
partial_contract.symbol = 'VIX'
partial_contract.secType = 'FUT'
partial_contract.lastTradeDateOrContractMonth = '202005'
partial_contract.currency = 'USD'

#ct_app.match_contract(partial_contract)
matching_contracts = ct_app.get_all_matching_contracts(partial_contract)
matching_contracts

[4379906232: 380797931,VIX,FUT,20200520,0.0,,1000,CFE,,USD,VXK0,VX,False,,combo:,
 4587099472: 414547673,VIX,FUT,20200506,0.0,,1000,CFE,,USD,VX18K0,VX18,False,,combo:,
 4379761688: 415634831,VIX,FUT,20200513,0.0,,1000,CFE,,USD,VX19K0,VX19,False,,combo:,
 4587327840: 416528422,VIX,FUT,20200527,0.0,,1000,CFE,,USD,VX21K0,VX21,False,,combo:]

In [5]:
_contract = matching_contracts[0]
#ct_app.add_to_saved_contracts([contract])
#ct_app.get_contract(contract.localSymbol)

In [16]:
ticker = 'VXK0'
frequency = '1s'

start = datetime.datetime(2020, 3, 17, 0, 0, 0)
end = datetime.datetime(2020, 3, 21, 0, 0, 0)

# Get the contract for the target ticker
contract = ct_app.get_contract(ticker)
download_and_save_daily_data(md_app, contract, start, end, skip_weekends=True, min_count=1500)

Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 138.70131812095642 seconds to avoid pacing violation on total requests.
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 260.9269650936127 seconds to avoid pacing violation on total requests.
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 5.821851110458374 seconds to avoid pacing violation on total requests.
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
0: 0
Finished downloading for 20200317 00:00:00. Time elapsed: 5.003540992736816
Saved data to data/VXK0_1s_20200317.csv
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid

KeyboardInterrupt: 

In [29]:
# Create a time series request
ticker = 'VXK0'
frequency = '1s'
start = '20200320 00:00:00'
end = '20200321 00:00:00'
duration = ''

contractList = [ct_app.get_contract(ticker)]
reqObjList = md_app.create_historical_data_request(contractList, is_snapshot=True, frequency=frequency, 
                                        use_rth=False, data_type="TRADES",
                                        start=start, end=end, duration=duration)

In [30]:
# Place the time series request
reqObj = reqObjList[0]
reqObj.place_request()

Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 239.9515830039978 seconds to avoid pacing violation on total requests.
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping 294.08257994651797 seconds to avoid pacing violation on total requests.
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...
Sleeping to avoid pacing violation from requests on same contract...


In [31]:
# Wait until the data has been returned by IB
data = reqObj.get_data()
min_count = 1000

n_requests = len(reqObj.get_subrequests())
t0 = time.time()
ct = 0
while ct != 48:
    print(ct)
    time.sleep(5)
    req_lengths = [len(data[idx][0]) for idx in range(n_requests)]    
    ct = sum([min_count <= r for r in req_lengths])

print('Finished. Time elapsed: {}'.format(time.time() - t0))

0
Finished. Time elapsed: 5.003296852111816


In [32]:
# Turn the data into a dataframe
df = pd.DataFrame()
for d in data:
    df = pd.concat([df, pd.DataFrame(d[0])])
    
# Drop duplicated rows
df.drop_duplicates(inplace=True)
df = df[df.volume > 0]
df.shape

(8819, 8)

In [33]:
# Save to file
date = start.split(' ')[0]
symbol = reqObj.contract.localSymbol
filename = 'data/{}_{}_{}.csv'.format(symbol, reqObj.frequency, date)
df.to_csv(filename, index=False)
print(filename)

data/VXK0_1s_20200320.csv


In [ ]:
# Compare the size of the various files to one another
for f in sorted(os.listdir('data')):
    tmp = pd.read_csv('data/' + f)
    tmp.drop_duplicates(inplace=True)
    n_zeros = np.sum(tmp.volume.values == 0)
    N = tmp.shape[0]
    print('{}: {} {}'.format(f, N, N - n_zeros))